# Begginer level tf

## Imports

In [3]:
import pandas as pd
import tensorflow as tf
import keras_nlp
from sklearn.model_selection import train_test_split

E0000 00:00:1726614927.629268    2415 common_lib.cc:798] Could not set metric server port: INVALID_ARGUMENT: Could not find SliceBuilder port 8471 in any of the 0 ports provided in `tpu_process_addresses`="local"
=== Source Location Trace: === 
learning/45eac/tfrc/runtime/common_lib.cc:479
D0917 23:15:27.637767293    2415 config.cc:196]                        gRPC EXPERIMENT call_status_override_on_cancellation   OFF (default:OFF)
D0917 23:15:27.637781134    2415 config.cc:196]                        gRPC EXPERIMENT call_v3                                OFF (default:OFF)
D0917 23:15:27.637784508    2415 config.cc:196]                        gRPC EXPERIMENT canary_client_privacy                  ON  (default:ON)
D0917 23:15:27.637786836    2415 config.cc:196]                        gRPC EXPERIMENT capture_base_context                   ON  (default:ON)
D0917 23:15:27.637789233    2415 config.cc:196]                        gRPC EXPERIMENT client_idleness                        ON  (defa

In [4]:
try:
    # detect and init the TPU
    resolver = tf.distribute.cluster_resolver.TPUClusterResolver()
    tf.config.experimental_connect_to_cluster(resolver)
    tf.tpu.experimental.initialize_tpu_system(resolver)
    strategy = tf.distribute.TPUStrategy(resolver)
    print("All devices: ", tf.config.list_logical_devices('TPU'))
except ValueError:
    strategy = tf.distribute.get_strategy()  # default strategy if no TPU available

INFO:tensorflow:Deallocate tpu buffers before initializing tpu system.
INFO:tensorflow:Initializing the TPU system: local


I0000 00:00:1726614933.953678    2415 service.cc:145] XLA service 0x56e8e5525610 initialized for platform TPU (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1726614933.953753    2415 service.cc:153]   StreamExecutor device (0): TPU, 2a886c8
I0000 00:00:1726614933.953757    2415 service.cc:153]   StreamExecutor device (1): TPU, 2a886c8
I0000 00:00:1726614933.953762    2415 service.cc:153]   StreamExecutor device (2): TPU, 2a886c8
I0000 00:00:1726614933.953764    2415 service.cc:153]   StreamExecutor device (3): TPU, 2a886c8
I0000 00:00:1726614933.953767    2415 service.cc:153]   StreamExecutor device (4): TPU, 2a886c8
I0000 00:00:1726614933.953769    2415 service.cc:153]   StreamExecutor device (5): TPU, 2a886c8
I0000 00:00:1726614933.953772    2415 service.cc:153]   StreamExecutor device (6): TPU, 2a886c8
I0000 00:00:1726614933.953775    2415 service.cc:153]   StreamExecutor device (7): TPU, 2a886c8


INFO:tensorflow:Finished initializing TPU system.
INFO:tensorflow:Found TPU system:
INFO:tensorflow:*** Num TPU Cores: 8
INFO:tensorflow:*** Num TPU Workers: 1
INFO:tensorflow:*** Num TPU Cores Per Worker: 8
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:CPU:0, CPU, 0, 0)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU:0, TPU, 0, 0)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU:1, TPU, 0, 0)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU:2, TPU, 0, 0)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU:3, TPU, 0, 0)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU:4, TPU, 0, 0)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU:5, TPU, 0, 0)
I

In [5]:
training_data = pd.read_csv('/kaggle/input/contradictory-my-dear-watson/train.csv')

df_train, df_test = train_test_split(training_data, test_size=0.2, random_state=42)

## Model

In [6]:
import tensorflow as tf
import keras_nlp

# Asegúrate de que 'premise' y 'hypothesis' sean listas de strings
premises = df_train['premise'].astype(str).tolist()
hypotheses = df_train['hypothesis'].astype(str).tolist()

# Define las labels (etiquetas) para entrenar
labels = tf.convert_to_tensor(df_train['label'])

# Verifica que todas las entradas tengan la misma longitud
assert len(premises) == len(hypotheses) == len(labels), "Las dimensiones de las entradas no coinciden."

# Crea un dataset a partir de las entradas
train_dataset = tf.data.Dataset.from_tensor_slices(((premises, hypotheses), labels)).batch(32)

# Define el modelo dentro del scope de la estrategia
with strategy.scope():
    classifier = keras_nlp.models.DistilBertClassifier.from_preset(
        "distil_bert_base_multi",
        num_classes=3
    )
    classifier.compile(
        optimizer=tf.keras.optimizers.Adam(),
        loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
        metrics=['accuracy']
    )

# Entrena el modelo con el dataset
classifier.fit(train_dataset, epochs=3)


I0000 00:00:1726614943.261491    2415 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


Epoch 1/3


2024-09-17 23:16:20.909472: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:961] model_pruner failed: INVALID_ARGUMENT: Graph does not contain terminal node StatefulPartitionedCall.
I0000 00:00:1726614982.241944    3214 tpu_compilation_cache_interface.cc:441] TPU host compilation cache miss: cache_key(3837ee7b802ff347:0:0), session_name()
I0000 00:00:1726615033.703310    3214 tpu_compile_op_common.cc:245] Compilation of 3837ee7b802ff347:0:0 with session name  took 51.461325571s and succeeded
I0000 00:00:1726615033.840195    3214 tpu_compilation_cache_interface.cc:475] TPU host compilation cache: compilation complete for cache_key(3837ee7b802ff347:0:0), session_name(), subgraph_key(std::string(property.function_name) = "cluster_one_step_on_iterator_4784323208619478065", property.function_library_fingerprint = 7487323565412563007, property.mlir_module_fingerprint = 0, property.num_replicas = 8, topology.chip_bounds().x = 2, topology.chip_bounds().y = 2, topology.chip_bounds().z =

303/303 ━━━━━━━━━━━━━━━━━━━━ 87s 83ms/step - accuracy: 0.3588 - loss: 1.0973
Epoch 2/3
303/303 ━━━━━━━━━━━━━━━━━━━━ 26s 83ms/step - accuracy: 0.3510 - loss: 1.0984
Epoch 3/3
303/303 ━━━━━━━━━━━━━━━━━━━━ 26s 83ms/step - accuracy: 0.3382 - loss: 1.1004


In [7]:
premises = df_test['premise'].astype(str).tolist()
hypotheses = df_test['hypothesis'].astype(str).tolist()

# Define las labels (etiquetas) para entrenar
labels = tf.convert_to_tensor(df_test['label'])

test_dataset = tf.data.Dataset.from_tensor_slices(((premises, hypotheses), labels)).batch(32)

test_loss, test_accuracy = classifier.evaluate(test_dataset)
print(f'Precisión en conjunto de test: {test_accuracy:.4f}')

2024-09-17 23:18:37.282868: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:961] model_pruner failed: INVALID_ARGUMENT: Graph does not contain terminal node Add/ReadVariableOp.
I0000 00:00:1726615117.663437    3232 tpu_compilation_cache_interface.cc:441] TPU host compilation cache miss: cache_key(e6da268813c0e728:0:0), session_name()


 7/76 ━━━━━━━━━━━━━━━━━━━━ 1s 20ms/step - accuracy: 0.4070 - loss: 1.0983

I0000 00:00:1726615119.921176    3232 tpu_compile_op_common.cc:245] Compilation of e6da268813c0e728:0:0 with session name  took 2.257695412s and succeeded
I0000 00:00:1726615119.938844    3232 tpu_compilation_cache_interface.cc:475] TPU host compilation cache: compilation complete for cache_key(e6da268813c0e728:0:0), session_name(), subgraph_key(std::string(property.function_name) = "cluster_one_step_on_iterator_8638409636061188340", property.function_library_fingerprint = 12571208160766893425, property.mlir_module_fingerprint = 0, property.num_replicas = 8, topology.chip_bounds().x = 2, topology.chip_bounds().y = 2, topology.chip_bounds().z = 1, topology.wrap().x = false, topology.wrap().y = false, topology.wrap().z = false, std::string(property.shapes_prefix) = "4,512,;4,512,;4,;", property.guaranteed_constants_size = 0, embedding_partitions_fingerprint = "1688352644216761960")
I0000 00:00:1726615119.938887    3232 tpu_compilation_cache_interface.cc:541] After adding entry for key e6

73/76 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - accuracy: 0.3447 - loss: 1.0980

I0000 00:00:1726615121.419936    3187 tpu_compilation_cache_interface.cc:441] TPU host compilation cache miss: cache_key(deaff29a7385b01e:0:0), session_name()


76/76 ━━━━━━━━━━━━━━━━━━━━ 9s 52ms/step - accuracy: 0.3441 - loss: 1.0980
Precisión en conjunto de prueba: 0.3366


I0000 00:00:1726615123.841357    3187 tpu_compile_op_common.cc:245] Compilation of deaff29a7385b01e:0:0 with session name  took 2.421386158s and succeeded
I0000 00:00:1726615123.854607    3187 tpu_compilation_cache_interface.cc:475] TPU host compilation cache: compilation complete for cache_key(deaff29a7385b01e:0:0), session_name(), subgraph_key(std::string(property.function_name) = "cluster_one_step_on_iterator_8638409636061188340", property.function_library_fingerprint = 12571208160766893425, property.mlir_module_fingerprint = 0, property.num_replicas = 8, topology.chip_bounds().x = 2, topology.chip_bounds().y = 2, topology.chip_bounds().z = 1, topology.wrap().x = false, topology.wrap().y = false, topology.wrap().z = false, std::string(property.shapes_prefix) = "3,512,;3,512,;3,;", property.guaranteed_constants_size = 0, embedding_partitions_fingerprint = "1688352644216761960")
I0000 00:00:1726615123.854636    3187 tpu_compilation_cache_interface.cc:541] After adding entry for key de